# DATA PREPROCESSING

In [22]:
# Loading the required libraries
import pandas as pd
import numpy as np

In [23]:
#Loading the data
df1 = pd.read_csv(
    '/Users/sunilravilla/Documents/Best Buy/Hackathon Data_csv.csv')
df1.tail()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS
846627,67,11/16/20,DRONE ACCESSORY,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,16.99,?,16,Fully-Stocked,0
846628,482,5/6/21,MOBILE PHONE MEDIA,FLASH MEMORY,ML - DI Accessories,VP - Digital Imaging,17.99,?,?,Fully-Stocked,0
846629,384,9/25/21,SO CAR ACCESSORIES,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,16.99,?,15.72,Fully-Stocked,3
846630,113,11/8/18,CAMERA PRO PRODUCTS,DIGITAL CAMCORDERS,ML - P&S and Action Cams and Drones,VP - Digital Imaging,"1,899.99",?,?,Fully-Stocked,0
846631,101,3/14/21,OTR MIC,OTR MICROWAVE,ML - Kitchen,VP - Major Appliances,619.99,?,549.99,Moderate,6


In [24]:
# Creating new columns PROMO_PRICE_AVBL and COMPETITOR_PRICE_AVBL
df1['PROMO_PRICE_AVBL'] = df1['PROMO_PRICE'].apply(
    lambda x: 0 if x == '?' else 1)
df1['COMPETITOR_PRICE_AVBL'] = df1['COMPETITOR_PRICE'].apply(
    lambda x: 0 if x == '?' else 1)

#Assigning the value of RETAIL_PRICE to PROMO_PRICE if PROMO_PRICE is '?' and COMPETITOR_PRICE is '?'
df1['PROMO_PRICE']=df1.apply(lambda x: x['RETAIL_PRICE'] if x['PROMO_PRICE']=='?' else x['PROMO_PRICE'],axis=1)
df1['COMPETITOR_PRICE'] = df1.apply(
    lambda x: x['RETAIL_PRICE'] if x['COMPETITOR_PRICE'] == '?' else x['COMPETITOR_PRICE'], axis=1)


#Removing the ',' from the values of numeric columns
numeric_cols = ['PROMO_PRICE', 'RETAIL_PRICE', 'COMPETITOR_PRICE', 'DAILY_UNITS']
for col in numeric_cols:
    df1[col] = df1[col].apply(lambda x: x.replace(',', '') if type(x) == str else x)
    df1[col] = df1[col].astype(float)

# Converting  SUBCLASS_NAME ,CLASS_NAME, ML_NAME, CATEGORY_NAME to string
string_cols = ['SUBCLASS_NAME', 'CLASS_NAME', 'ML_NAME', 'CATEGORY_NAME']
for col in string_cols:
    df1[col] = df1[col].astype(str)

# coverting the data type of SALES_DATE to datetime
df1['SALES_DATE'] = pd.to_datetime(df1['SALES_DATE'])

# Sorting the data by Encoded_SKU_ID and SALES_DATE
df1 = df1.sort_values(
    by=['Encoded_SKU_ID', 'SALES_DATE']).reset_index(drop=True)
df1.head(10)


,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,PROMO_PRICE_AVBL,COMPETITOR_PRICE_AVBL
0,1,2017-05-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0,0
1,1,2017-05-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0,0
2,1,2017-05-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0,0
3,1,2017-05-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0,0
4,1,2017-06-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0,0
5,1,2017-06-02,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,4.0,0,0
6,1,2017-06-03,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,7.0,0,0
7,1,2017-06-04,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,4.0,0,0
8,1,2017-06-05,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,2.0,0,0
9,1,2017-06-06,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,4.0,0,0


In [25]:
#Adding the missing dates for each SKU_ID, after the first occurence of the SKU_ID and before the last occurence of the SKU_ID
df1 = df1.groupby('Encoded_SKU_ID').apply(
    lambda x: x.set_index('SALES_DATE').asfreq('D')
).drop('Encoded_SKU_ID', axis=1).reset_index()

df1 = df1.sort_values(
    by=['Encoded_SKU_ID', 'SALES_DATE']).reset_index(drop=True)
df1.head()


,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,PROMO_PRICE_AVBL,COMPETITOR_PRICE_AVBL
0,1,2017-05-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0
1,1,2017-05-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0
2,1,2017-05-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0
3,1,2017-05-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0
4,1,2017-06-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0


In [26]:
#Filling the missing values
df1['SUBCLASS_NAME'] = df1['SUBCLASS_NAME'].fillna(method='ffill')
df1['CLASS_NAME'] = df1['CLASS_NAME'].fillna(method='ffill')
df1['ML_NAME'] = df1['ML_NAME'].fillna(method='ffill')
df1['CATEGORY_NAME'] = df1['CATEGORY_NAME'].fillna(method='ffill')
df1['PROMO_PRICE'] = df1['PROMO_PRICE'].fillna(method='ffill')
df1['RETAIL_PRICE'] = df1['RETAIL_PRICE'].fillna(method='ffill')
df1['COMPETITOR_PRICE'] = df1['COMPETITOR_PRICE'].fillna(method='ffill')
df1['Inventory'] = df1['Inventory'].fillna('Missing')
df1['COMPETITOR_PRICE_AVBL'] = df1['COMPETITOR_PRICE_AVBL'].fillna(0)
df1['PROMO_PRICE_AVBL'] = df1['PROMO_PRICE_AVBL'].fillna(0)
df1['DAILY_UNITS'] = df1['DAILY_UNITS'].fillna(0)


In [27]:
# Creating new columns 'MONTH','WEEK','DAY'
df1['MONTH'] = df1['SALES_DATE'].dt.month
df1['WEEK'] = df1['SALES_DATE'].dt.week
df1['DAY'] = df1['SALES_DATE'].dt.day_name()
df1.head(15)


/var/folders/lb/b10ttmcx6p96khnw319kfyth0000gn/T/ipykernel_92323/3405673048.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df1['WEEK'] = df1['SALES_DATE'].dt.week


,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,PROMO_PRICE_AVBL,COMPETITOR_PRICE_AVBL,MONTH,WEEK,DAY
0,1,2017-05-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,21,Sunday
1,1,2017-05-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Monday
2,1,2017-05-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Tuesday
3,1,2017-05-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Wednesday
4,1,2017-06-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,6,22,Thursday
5,1,2017-06-02,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,4.0,0.0,0.0,6,22,Friday
6,1,2017-06-03,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,7.0,0.0,0.0,6,22,Saturday
7,1,2017-06-04,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,4.0,0.0,0.0,6,22,Sunday
8,1,2017-06-05,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,2.0,0.0,0.0,6,23,Monday
9,1,2017-06-06,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,4.0,0.0,0.0,6,23,Tuesday


In [28]:
df1.dtypes


Encoded_SKU_ID                    int64
SALES_DATE               datetime64[ns]
SUBCLASS_NAME                    object
CLASS_NAME                       object
ML_NAME                          object
CATEGORY_NAME                    object
RETAIL_PRICE                    float64
PROMO_PRICE                     float64
COMPETITOR_PRICE                float64
Inventory                        object
DAILY_UNITS                     float64
PROMO_PRICE_AVBL                float64
COMPETITOR_PRICE_AVBL           float64
MONTH                             int64
WEEK                              int64
DAY                              object
dtype: object

In [29]:
df1 .isnull().sum()


Encoded_SKU_ID           0
SALES_DATE               0
SUBCLASS_NAME            0
CLASS_NAME               0
ML_NAME                  0
CATEGORY_NAME            0
RETAIL_PRICE             0
PROMO_PRICE              0
COMPETITOR_PRICE         0
Inventory                0
DAILY_UNITS              0
PROMO_PRICE_AVBL         0
COMPETITOR_PRICE_AVBL    0
MONTH                    0
WEEK                     0
DAY                      0
dtype: int64

In [30]:
df1.columns = df1.columns.str.lower()
df1.head(10)


,encoded_sku_id,sales_date,subclass_name,class_name,ml_name,category_name,retail_price,promo_price,competitor_price,inventory,daily_units,promo_price_avbl,competitor_price_avbl,month,week,day
0,1,2017-05-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,21,Sunday
1,1,2017-05-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Monday
2,1,2017-05-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Tuesday
3,1,2017-05-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Wednesday
4,1,2017-06-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,6,22,Thursday
5,1,2017-06-02,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,4.0,0.0,0.0,6,22,Friday
6,1,2017-06-03,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,7.0,0.0,0.0,6,22,Saturday
7,1,2017-06-04,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,4.0,0.0,0.0,6,22,Sunday
8,1,2017-06-05,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,2.0,0.0,0.0,6,23,Monday
9,1,2017-06-06,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,4.0,0.0,0.0,6,23,Tuesday


In [31]:
# Writing the preprocessed data to a csv file
df1.to_csv('Best_buy_Data_preprocessed.csv', index=False)


# THE END!!!